In [1]:
import flask
from flask import url_for
app = flask.Flask(__name__)

# 首页路由
@app.route('/')
def index():
    """
    首页路由函数
    
    参数: 无
    
    返回值: 返回字符串 'index'
    """
    return 'index'

# 登录路由
@app.route('/login')
def login():
    """
    登录路由函数
    
    参数: 无
    
    返回值: 返回字符串 'login'
    """
    return 'login'

# 用户个人资料路由
@app.route('/user/<username>')
def profile(username):
    """
    用户个人资料路由函数
    
    参数:
    - username: 用户名，字符串类型
    
    返回值: 返回格式化后的字符串，表示用户的个人资料页面
    """
    return f'{username}\'s profile'

# 使用测试请求上下文，测试生成的URL
with app.test_request_context():
    # 生成首页的URL
    print(url_for('index'))
    # 生成登录页面的URL
    print(url_for('login'))
    # 生成登录页面的URL，带next参数，用于登录后重定向
    print(url_for('login', next='/'))
    # 生成指定用户名的个人资料页面URL
    print(url_for('profile', username='John Doe'))




/
/login
/login?next=%2F
/user/John%20Doe


HTTP 方法
Web 应用使用不同的 HTTP 方法处理 URL 。当您使用 Flask 时，应当熟悉 HTTP 方法。缺省情况下，一个路由只回应 GET 请求。可以使用 route() 装饰器的 methods 参数来处理不同的 HTTP 方法。

In [2]:
import flask
from flask import request
app = flask.Flask(__name__)
@app.route('/login',methods=['GET','POST'])
def login():
    """
    处理用户登录请求。
    
    - 参数: 无
    - 返回值: 根据请求方法和认证结果，返回相应的页面或文本。
    """
    # 根据请求方法决定操作
    if request.method=='GET':
        # 如果是GET请求，返回登录页面
        return render_template('login.html')
    else:
        # 如果是POST请求，获取用户名和密码
        username=request.form.get('username')
        password=request.form.get('password')
        # 验证用户名和密码
        if username=='admin' and password=='123456':
            # 如果认证成功，返回成功信息
            return 'login success'
        else:
            # 如果认证失败，返回失败信息
            return 'login fail'

In [3]:
@app.get('/login')
def login_get():
    return show_the_login_form()

@app.post('/login')
def login_post():
    return do_the_login()



动态的 web 应用也需要静态文件，一般是 CSS 和 JavaScript 文件。理想情 况下您的服务器已经配置好了为您的提供静态文件的服务。但是在开发过程中， Flask 也能做好这项工作。只要在您的包或模块旁边创建一个名为 static 的文件夹就行了。静态文件位于应用的 /static 中。

使用特定的 'static' 端点就可以生成相应的 URL
这个静态文件在文件系统中的位置应该是 static/style.css 。

In [4]:
url_for('static',filename = 'style.css')


RuntimeError: Attempted to generate a URL without the application context being pushed. This has to be executed when application context is available.

渲染模板
在 Python 内部生成 HTML 不好玩，且相当笨拙。因为您必须自己负责 HTML 转义，以确保应用的安全。因此， Flask 自动为您配置 Jinja2 模板引擎。

模板可被用于生成任何类型的文本文件。对于 web 应用来说，主要用于生成 HTML 页面，但是也可以生成 markdown 、用于电子邮件的纯文本等等。

HTML 、 CSS 和其他 web API ，请参阅 MDN Web 文档 。

使用 render_template() 方法可以渲染模板，您只要提供模板 名称和需要作为参数传递给模板的变量就行了。下面是一个简单的模板渲染例 子:

In [ ]:
from flask import render_template
@app.route('/hello/')
@app.route('/hello/<name>')
def hello(name=None):
    return render_template('hello.html',name=name)


在模板内部可以像使用 url_for() 和 get_flashed_messages() 函数一样访问 config 、 request 、 session 和 g [1] 对象。

模板在继承使用的情况下尤其有用。其工作原理参见 模板继承 。简单的说，模板继承可以使每个页 面的特定元素（如页头、导航和页尾）保持一致。

自动转义默认开启。因此，如果 name 包含 HTML ，那么会被自动转义。 如果您可以信任某个变量，且知道它是安全的 HTML （例如变量来自一个把 wiki 标记转换为 HTML 的模块），那么可以使用 Markup 类把它标记为安全的，或者在模板中使用 |safe 过滤器。更多例子参见 Jinja 2 文档。   
模板继承

In [ ]:
from markupsafe import Markup
Markup('<strong>Hello %s!</strong>') % '<blink>hacker</blink>'
Markup('<strong>Hello &lt;blink&gt;hacker&lt;/blink&gt;!</strong>')
Markup.escape('<blink>hacker</blink>')
Markup('&lt;blink&gt;hacker&lt;/blink&gt;')
Markup('<em>Marked up</em> &raquo; HTML').striptags()
'Marked up » HTM

SyntaxError: unterminated string literal (detected at line 7) (2497682364.py, line 7)

本地环境
内部信息
如果您想了解工作原理和如何使用本地环境进行测试，那么请阅读本节， 否则可以跳过本节。

某些对象在 Flask 中是全局对象，但不是通常意义下的全局对象。这些对象实 际上是特定环境下本地对象的代理。真拗口！但还是很容易理解的。

设想现在处于处理线程的环境中。一个请求进来了，服务器决定生成一个新线 程（或者叫其他什么名称的东西，这个下层的东西能够处理包括线程在内的并 发系统）。当 Flask 开始其内部请求处理时会把当前线程作为活动环境，并把 当前应用和 WSGI 环境绑定到这个环境（线程）。它以一种聪明的方式使得一 个应用可以在不中断的情况下调用另一个应用。

这对您有什么用？基本上您可以完全不必理会。这个只有在做单元测试时才有 用。在测试时会遇到由于没有请求对象而导致依赖于请求的代码会突然崩溃的 情况。对策是自己创建一个请求对象并绑定到环境。最简单的单元测试解决方 案是使用 test_request_context() 环境管理器。通过使 用 with 语句可以绑定一个测试请求，以便于交互。例如:     from flask import current_app

In [ ]:
from flask import request
with app.test_request_context('/hello',method = 'POST'):    
    assert request.path =='/hello'
    assert request.method == 'POST'
    

请求对象

请求对象在 API 一节中有详细说明这里不细谈（参见 Request ）。这里简略地谈一下最常见的操作。首先，您必 须从 flask 模块导入请求对象:

from flask import request
通过使用 method 属性可以操作当前请求方法，通过 使用 form 属性处理表单数据（在 POST 或者 PUT 请求中传输的数据）。以下是使用上述两个属性的例子:

In [ ]:
@app.route('/login',methods=['POST','GET'])
def login():
    error = None
    if request.method =='POST':
        if vaild_login(request.form['username'],
                       request.form['password']):
            return log_the_user_in(request.form['username'])
        else:
            error = 'Invalid username/password'
            return render_template('login.html',error=error)

文件上传

用 Flask 处理文件上传很容易，只要确保不要忘记在您的 HTML 表单中设置 enctype="multipart/form-data" 属性就可以了。否则浏览器将不会传送 您的文件。

已上传的文件被储存在内存或文件系统的临时位置。您可以通过请求对象 files 属性来访问上传的文件。每个上传的文件都储 存在这个字典型属性中。这个属性基本和标准 Python file 对象一 样，另外多出一个用于把上传文件保存到服务器的文件系统中的 save() 方法。下例展示其如 何运作:

In [ ]:
from flask import request
@app.route('/upload',methods=['GET','POST'])
def upload_file():
    if request.method == 'POST':
        f = request.files['the_file']
        f.save('/var/www/uploads/uploaded_file.txt')

如果想要知道文件上传之前其在客户端系统中的名称，可以使用 filename 属性。但是请牢 记这个值是可以伪造的，永远不要信任这个值。如果想要把客户端的文件名作 为服务器上的文件名，可以通过 Werkzeug 提供的 secure_filename() 函数:

In [ ]:
from werzeug.utils import securee_filename
@app.route('/upload',method=['POST','GET'])
def upload_file():
    if request.method == 'POST':
        file = request.files['the_file']
        file.save(f"/var/www/uploads/{securee_filename(file.filename)}")
        return 'file uploaded successfully'

Cookies

要访问 cookies ，可以使用 cookies 属性。可以使 用响应对象 的 set_cookie 方法来设置 cookies 。 请求对象的 cookies 属性是一个包含了客户端传输 的所有 cookies 的字典。在 Flask 中，如果使用 会话 ，那么就 不要直接使用 cookies ，因为 会话 比较安全一些。

读取 cookies:

In [ ]:
from flask import request
@app.route('/')
def index():
    username = request.cookies.get('username')


In [ ]:
from flask import make_response
@app.route('/')
def index():
    resp = make_response(render_template(...))
    resp.set_cookie('username', 'the username')
    return resp

重定向和错误
使用 redirect() 函数可以重定向。使用 abort() 可以更早退出请求，并返回错误代码:

In [ ]:
from flask import abort, redirect,url_for
@app.route('/')
def index():
    return redirect(url_for('login'))
@app.route('/login')
def login():
    abort(401)
    this_is_never_executed()


上例实际上是没有意义的，它让一个用户从索引页重定向到一个无法访问的页 面（401 表示禁止访问）。但是上例可以说明重定向和出错跳出是如何工作的。


缺省情况下每种出错代码都会对应显示一个黑白的出错页面。使用 errorhandler() 装饰器可以定制出错页面:

In [ ]:
@app.errorhandler(404)
def page_not_found(error)
    return render_tempalte('page_not_found.html'),404
@app.errorhandler(500)
def internal_server_error(error)
    return render_template('internal_server_error.html'),500
@app.route('/')
def index():
    return render_template('index.html')

In [ ]:
from flask import Flask,render_template_string, make_response
app = Flask(__name__)
@app.errorhandfler(404)
def not_found(error):
    resp = make_response(render_template('ArithmeticError.html'),404)
    resp.headers['X=Something'] = 'A value'
    return resp



JSON 格式的 API
JSON 格式的响应是常见的，用 Flask 写这样的 API 是很容易上手的。如果从 视图返回一个 dict 或者 list ，那么它会被转换为一个 JSON 响应。

In [ ]:
@app.route('/me')
def me_api():
    user = get_current_user()
    return{
        "username":user.username,
        "theme":user.theme,
        "image":url_for("user_image",filename=user.image)

        
    }

## 会话
除了请求对象之外还有一种称为 session 的对象，允许您在 不同请求之间储存信息。这个对象相当于用密钥签名加密的 cookie ，即用户 可以查看您的 cookie ，但是如果没有密钥就无法修改它。

使用会话之前您必须设置一个密钥。举例说明:
在 Flask 中，会话（session）是一个非常重要的功能，用于在用户与 Web 应用之间的多个请求过程中保持状态信息。会话使得 Web 应用能够存储和检索关于用户状态的数据。这里的代码示例使用会话来追踪用户的登录状态，具体的作用和细节如下：

### 1. 存储用户信息
在用户登录时，会话被用来存储用户的标识信息（如用户名）。这样，即使在多个请求之间，服务器也可以记住用户的身份。这是通过在服务器端为每个会话生成唯一的会话ID，并将此ID存储在用户浏览器的cookie中实现的。服务器利用这个会话ID来检索会话数据。

### 2. 持续性跟踪
会话允许应用跟踪用户的登录状态，不需要用户在每次请求时重新进行身份验证。例如，在这段代码中：

当用户成功登录后，username 会被存储在会话中：
python
复制代码
session['username'] = request.form['username']
这个操作把用户的用户名存入会话，允许跨请求持续访问这些信息。
当用户访问其他页面（比如首页）时，应用可以检查会话中是否有 username 来判断用户是否已经登录：
python
复制代码
if 'username' in session:
    return f'Logged in as {session["username"]}'
### 3. 安全和隐私
使用会话时，Flask 会自动处理安全性，例如通过加密cookie来保护会话数据。app.secret_key 用于签名会话cookie，确保会话cookie没有被篡改。

### 4. 登出和清理会话
当用户选择登出时，会话中存储的用户信息需要被清除，以确保用户的登录状态被正确地重置。这是通过 session.pop 方法完成的，该方法从会话中移除特定的键值对：

python
复制代码
session.pop('username', None)
总结
会话为无状态的 HTTP 协议提供了一种持久状态的机制，非常适合用于实现用户认证、购物车数据存储等功能。在这个 Flask 应用中，会话主要用于维护用户的登录状态，在用户的多个请求之间共享用户名信息。这种方式简单直接，适合小到中等规模的应用，但在设计时需要考虑到安全性和数据保护的相关策略。

In [ ]:
from flask import session
app.secrete_key = 'secret'

@app.route('/')
def index():
    if 'username' in session:
        return f'logged in as {session["username"]}'
    return 'You aer not logged in'
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method =='POST':
        session['username'] = request.form['username']
        return redirect(url_for('index'))
    return '''
        <form method="post">
            <p><input type=text name=username>
            <p><input type=submit value=Login>
        </form>
    '''
@app.route('/logout')
def logout():
    session.pop('username',None)
    return redirect(url_for('index'))
